In [7]:
import os
import rasterio as rio
import numpy as np
import geopandas as gpd
import pandas as pd
from osgeo import gdal
import matplotlib.pyplot as plt
from progressbar import ProgressBar

from utils import *

In [8]:
plt.rcParams["figure.figsize"] = (28,10)
plt.ion()

In [9]:
data_dir = 'data_100m'

dem_file            = f'{data_dir}/dem.tif'
slope_file          = f'{data_dir}/slope.tif'
aspect_file         = f'{data_dir}/aspect.tif'
northing_file       = f'{data_dir}/northing.tif'
easting_file        = f'{data_dir}/easting.tif'

urban_distance_file = f'{data_dir}/urban_distance.tiff'
roads_distance_file = f'{data_dir}/roads_distance.tiff'
tracks_distance_file= f'{data_dir}/tracks_distance.tiff'
crops_distance_file = f'{data_dir}/crops_distance.tiff'

parks_raster_file = f'{data_dir}/parks.tiff'

vegetation_agg_file = f'{data_dir}/vegetation_agg.tiff'
vegetation_file     = f'{data_dir}/vegetation.tiff'
vegetation_type_file     = f'{data_dir}/vegetation_type.tiff'


fires_shp           = 'shapefiles/perimetrazioni_1997_2017.shp'
vegetation_shp      = 'shapefiles/tipiforestali_marj.shp'
roads_shp           = 'shapefiles/Tratte_stradali.shp'
tracks_shp          = 'shapefiles/Rete_Escursionistica_Ligure.shp'
parks_shp           = 'shapefiles/parchi_sic_zps_zonerilevanti2.shp'

In [10]:
with rio.open(dem_file) as src:
    print(f'Reading dem file {dem_file}')
    dem = src.read(1)
    dem[dem <= -9999] = np.NaN
_, dx, _, _, _, dy = src.transform.to_gdal()    
bbox = src.bounds

Reading dem file data_100m/dem.tif


## PREPROCESSING

In [11]:
with rio.open(dem_file) as src:
    print(f'Reading dem file {dem_file}')
    dem = src.read(1)
    dem[dem <= -9999] = np.NaN

if not os.path.isfile(slope_file):
    print(f'Creating slope file {slope_file}')
    gdal.DEMProcessing(slope_file, dem_file, 'slope')

    
if not os.path.isfile(northing_file) or not os.path.isfile(easting_file):

    if not os.path.isfile(aspect_file):
        print(f'Creating aspect file {aspect_file}')
        gdal.DEMProcessing(aspect_file, dem_file, 'aspect')

    with rio.open(aspect_file) as f:
        print(f'Calculating northing and easting files')
        print(f'Reading aspect file {aspect_file}')
        aspect = f.read(1)
        aspect[aspect <= -9999] = np.NaN    
        northing = np.cos(aspect * np.pi/180.0)
        easting = np.sin(aspect * np.pi/180.0)

    print(f'Saving northing file {northing_file}')
    save_raster_as(northing, northing_file, aspect_file)
    print(f'Saving easting file {easting_file}')    
    save_raster_as(easting, easting_file, aspect_file)

Reading dem file data_100m/dem.tif


In [12]:
print(f'Reading vegetation shp {vegetation_shp}')
vegetation = gpd.read_file(vegetation_shp)
vegetation['encoded_decodifica'], mapping = encode_feature(vegetation, 'Decodifica') 
vegetation['encoded_tipologia'], mapping_tipologia = encode_feature(vegetation, 'Tipologia')

if not os.path.isfile(vegetation_file):
    print(f'Rasterizing vegetation')
    vegetation_raster = rasterize_numerical_feature(vegetation, dem_file, 'encoded_decodifica')
    print(f'Writing vegetation_raster file {vegetation_file}')    
    save_raster_as(vegetation_raster, vegetation_file, dem_file)
    

if not os.path.isfile(vegetation_agg_file):
    print(f'Rasterizing vegetation_agg')    
    vegetation_agg_raster = rasterize_numerical_feature(vegetation, dem_file, 'id_agg_fv')
    print(f'Writing vegetation_agg_raster file {vegetation_agg_file}')
    save_raster_as(vegetation_agg_raster, vegetation_agg_file, dem_file)

    
if not os.path.isfile(vegetation_type_file):
    print(f'Rasterizing vegetation_type')    
    vegetation_type = rasterize_numerical_feature(vegetation, dem_file, 'encoded_tipologia')
    print(f'Writing vegetation_agg_new_raster file {vegetation_type_file}')
    save_raster_as(vegetation_type, vegetation_type_file, dem_file)
    


Reading vegetation shp shapefiles/tipiforestali_marj.shp


In [13]:
vegetation_agg_raster = read_tiff(vegetation_agg_file)
mask = vegetation_agg_raster >= 20
indices = np.argwhere(mask)
coordinates = extract_coordinates(indices, src)


Reading file data_100m/vegetation_agg.tiff


In [14]:
if not os.path.isfile(crops_distance_file):
    crops = vegetation.query(f'Coltivo == 1')
    crops_raster_file = f'{data_dir}/crops.tiff'
    crops_raster = rasterize_numerical_feature(crops, dem_file)
    save_raster_as(crops_raster, crops_raster_file, dem_file)

    write_distance_raster(crops_raster_file, crops_distance_file)

In [15]:
if not os.path.isfile(urban_distance_file):
    urban = vegetation.query(f'Urbano == 1')
    urban_raster_file = f'{data_dir}/urban.tiff'
    urban_raster = rasterize_numerical_feature(urban, dem_file)
    save_raster_as(urban_raster, urban_raster_file, dem_file)

    urban_distance_file = f'{data_dir}/urban_distance.tiff'            
    write_distance_raster(urban_raster_file, urban_distance_file)

In [16]:
if not os.path.isfile(roads_distance_file):
    print(f'Reading roads shp {roads_shp}')
    roads = gpd.read_file(roads_shp)

    roads_raster = rasterize_numerical_feature(roads, dem_file)
    roads_raster_file = f'{data_dir}/roads.tiff'
    save_raster_as(roads_raster, roads_raster_file, dem_file)

    write_distance_raster(roads_raster_file, roads_distance_file)
    

In [17]:
if not os.path.isfile(tracks_distance_file):
    print(f'Reading tracks shp {tracks_shp}')
    tracks = gpd.read_file(tracks_shp)

    tracks_raster = rasterize_numerical_feature(tracks, dem_file)
    tracks_raster_file = f'{data_dir}/tracks.tiff'
    save_raster_as(tracks_raster, tracks_raster_file, dem_file)

    write_distance_raster(tracks_raster_file, tracks_distance_file)
    

In [18]:
if not os.path.isfile(parks_raster_file):
    print(f'Reading parks shp {parks_shp}')
    parks = gpd.read_file(parks_shp)

    parks_raster = rasterize_numerical_feature(parks, dem_file)
    save_raster_as(parks_raster, parks_raster_file, dem_file)

## DATA PREPARATION

In [19]:
print(f'Reading northing file {slope_file}')
slope = read_tiff(slope_file)

print(f'Reading northing file {northing_file}')
northing = read_tiff(northing_file)
    
print(f'Reading easting file {easting_file}') 
easting = read_tiff(easting_file)

print(f'Reading vegetation_raster file {vegetation_file}')            
vegetation_raster = read_tiff(vegetation_file)
    
print(f'Reading vegetation_agg_raster file {vegetation_agg_file}')          
vegetation_agg_raster = read_tiff(vegetation_agg_file)

print(f'Reading vegetation_type file {vegetation_type_file}')          
vegetation_type = read_tiff(vegetation_type_file)

print(f'Reading urban distance file {urban_distance_file}')
urban_distance = read_tiff(urban_distance_file)

print(f'Reading roads distance file {roads_distance_file}')
roads_distance = read_tiff(roads_distance_file)

print(f'Reading crops distance file {crops_distance_file}')
crops_distance = read_tiff(crops_distance_file)

print(f'Reading tracks distance file {tracks_distance_file}')
tracks_distance = read_tiff(tracks_distance_file)

print(f'Reading parks file {parks_raster_file}')
parks_raster = read_tiff(parks_raster_file)

Reading northing file data_100m/slope.tif
Reading file data_100m/slope.tif
Reading northing file data_100m/northing.tif
Reading file data_100m/northing.tif
Reading easting file data_100m/easting.tif
Reading file data_100m/easting.tif
Reading vegetation_raster file data_100m/vegetation.tiff
Reading file data_100m/vegetation.tiff
Reading vegetation_agg_raster file data_100m/vegetation_agg.tiff
Reading file data_100m/vegetation_agg.tiff
Reading vegetation_type file data_100m/vegetation_type.tiff
Reading file data_100m/vegetation_type.tiff
Reading urban distance file data_100m/urban_distance.tiff
Reading file data_100m/urban_distance.tiff
Reading roads distance file data_100m/roads_distance.tiff
Reading file data_100m/roads_distance.tiff
Reading crops distance file data_100m/crops_distance.tiff
Reading file data_100m/crops_distance.tiff
Reading tracks distance file data_100m/tracks_distance.tiff
Reading file data_100m/tracks_distance.tiff
Reading parks file data_100m/parks.tiff
Reading fil

In [20]:
print(f'Reading vegetation shp {vegetation_shp}')
vegetation = gpd.read_file(vegetation_shp)
vegetation['encoded_decodifica'], mapping = encode_feature(vegetation, 'Decodifica')
vegetation['encoded_tipologia'], mapping_tipologia = encode_feature(vegetation, 'Tipologia')

Reading vegetation shp shapefiles/tipiforestali_marj.shp


In [21]:
# masking vegetation raster excluding non-vegetated areas
mask = (vegetation_agg_raster >= 20) & (vegetation_type != 19) & (dem != -9999) & (slope != -9999) & (northing != -9999) & (easting != -9999)
indeces = np.argwhere(mask)
coordinates = extract_coordinates(indeces, src)
points_geom = [Point(*p) for p in coordinates.T]

points = gpd.GeoDataFrame(pd.DataFrame(indeces, columns=['row', 'col']), geometry=points_geom, crs={'init': 'epsg:3003'})
points['x'] = points.geometry.x
points['y'] = points.geometry.y



In [22]:
print(f'Reading fires shp {fires_shp}')
fires = gpd.read_file(fires_shp)

points_envelopes = points.copy()
points_envelopes.geometry = points.geometry.buffer(dx/2).envelope

presences = gpd.sjoin(points_envelopes, fires)\
             .loc[:, ('x', 'y', 'index_right', 'data', 'anno', 'stagione', 'area_ha')]

presences.index.name = 'point_index'
presences.rename(columns={'index_right': 'fire_index'}, inplace=True)

presences.to_csv(f'{data_dir}/fires.csv')

Reading fires shp shapefiles/perimetrazioni_1997_2017.shp


### Extract veg type density for each point neighborood

In [23]:
from scipy import signal
from scipy import misc

window_size = 2
M = vegetation_type
types = np.unique(vegetation_type)

types_presence = {}

counter = np.ones((window_size*2+1, window_size*2+1))
counter[window_size, window_size] = 0
counter = counter / np.sum(counter)

bar = ProgressBar()
for t in bar(types):
    density_entry = 'perc_' + mapping_tipologia[t].replace(' ', '_').lower()
    types_presence[density_entry] = 100 * signal.convolve2d(M==t, counter, boundary='fill', mode='same')
    

100% (37 of 37) |########################| Elapsed Time: 0:00:07 Time:  0:00:07


In [25]:
veg_freq = np.array(list(types_presence.values())).argmax(0)

### Merge datasets to points.csv file

In [32]:
data_dict = {
    'dem': dem,
    'slope': slope,
    'north': northing,
    'east': easting,
    'veg': vegetation_raster,
    'veg_agg': vegetation_agg_raster,
    'veg_type': vegetation_type,
    'urban_d': urban_distance,
    'roads_d': roads_distance,
    'crops_d': crops_distance,
    'park': parks_raster,
    'tracks_d': tracks_distance,
    
    'veg_freq': veg_freq
}
data_dict.update(types_presence)

mappings = {
    'veg': mapping,
    'veg_type': mapping_tipologia,
    'veg_freq': mapping_tipologia,
}

bar = ProgressBar()
for k, v in bar(data_dict.items()):
    points[k] = v[points.row.values, points.col.values]
    if k in mappings:
        points[k] = points[k].apply(lambda v: mappings[k][v])
        

100% (50 of 50) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [33]:
points.loc[points['north'].isna(), 'north'] = 0
points.loc[points['east'].isna(), 'east'] = 0
points.loc[points['roads_d'].isna(), 'roads_d'] = -9999   

points.geometry = points.loc[points.index].geometry
points.index.name = 'point_index'

points_csv = points.copy()
points_csv['x'] = points_csv.geometry.x
points_csv['y'] = points_csv.geometry.y
points_csv = points_csv.drop('geometry', axis=1)

points_csv[['x', 'y', 'row', 'col'] + list(data_dict.keys())]

points_csv.to_csv(f'{data_dir}/points.csv')

# points.to_file(f'{data_dir}/points.geojson', driver='GeoJSON')